## Part 2


In [80]:
using Pkg

In [81]:
using CSV, DataFrames, StatsModels, GLM, Random, MLBase, MLJ, PrettyTables, FixedEffectModels

1. 

In [82]:
data= CSV.read(
    download("https://github.com/d2cml-ai/CausalAI-Course/raw/refs/heads/main/Labs/Assignment/Assignment_5/data/observational/biased_control.csv"),
    DataFrame);

In [83]:
data

Row,treat,age,educ,black,hisp,marr,nodegree,re74,re75,re78,agesq,agecube,educsq,u74,u75,interaction1,re74sq,re75sq
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,0,45,11,0,0,1,1,21.5167,25.2436,25.5647,2025,91125,121,0,0,236.683,462.967,637.237
2,0,21,14,0,0,0,0,3.17597,5.85256,13.4961,441,9261,196,0,0,44.4636,10.0868,34.2525
3,0,38,12,0,0,1,0,23.039,25.1308,25.5647,1444,54872,144,0,0,276.468,530.796,631.555
4,0,48,6,0,0,1,1,24.9944,25.2436,25.5647,2304,110592,36,0,0,149.966,624.718,637.237
5,0,18,8,0,0,1,1,1.6693,10.7276,9.86087,324,5832,64,0,0,13.3544,2.78655,115.082
6,0,22,11,0,0,1,1,16.3658,18.4493,25.5647,484,10648,121,0,0,180.023,267.838,340.376
7,0,48,10,0,0,1,1,16.8046,16.3546,18.0593,2304,110592,100,0,0,168.046,282.396,267.473
8,0,18,11,0,0,0,1,1.14421,3.62003,15.7393,324,5832,121,0,0,12.5863,1.30922,13.1046
9,0,48,9,0,0,1,1,25.8623,25.2436,25.5647,2304,110592,81,0,0,232.761,668.86,637.237


2. 

Spliting 

In [84]:
treatment= filter(row -> row.treat == 1, data)

Row,treat,age,educ,black,hisp,marr,nodegree,re74,re75,re78,agesq,agecube,educsq,u74,u75,interaction1,re74sq,re75sq
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,1,37,11,1,0,1,1,0.0,0.0,9.93005,1369,50653,121,1,1,0.0,0.0,0.0
2,1,22,9,0,1,0,1,0.0,0.0,3.59589,484,10648,81,1,1,0.0,0.0,0.0
3,1,30,12,1,0,0,0,0.0,0.0,24.9094,900,27000,144,1,1,0.0,0.0,0.0
4,1,27,11,1,0,0,1,0.0,0.0,7.50615,729,19683,121,1,1,0.0,0.0,0.0
5,1,33,8,1,0,0,1,0.0,0.0,0.28979,1089,35937,64,1,1,0.0,0.0,0.0
6,1,22,9,1,0,0,1,0.0,0.0,4.05649,484,10648,81,1,1,0.0,0.0,0.0
7,1,23,12,1,0,0,0,0.0,0.0,0.0,529,12167,144,1,1,0.0,0.0,0.0
8,1,32,11,1,0,0,1,0.0,0.0,8.47216,1024,32768,121,1,1,0.0,0.0,0.0
9,1,22,16,1,0,0,0,0.0,0.0,2.16402,484,10648,256,1,1,0.0,0.0,0.0


In [85]:
control = filter(row -> row.treat == 0, data)

Row,treat,age,educ,black,hisp,marr,nodegree,re74,re75,re78,agesq,agecube,educsq,u74,u75,interaction1,re74sq,re75sq
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,0,45,11,0,0,1,1,21.5167,25.2436,25.5647,2025,91125,121,0,0,236.683,462.967,637.237
2,0,21,14,0,0,0,0,3.17597,5.85256,13.4961,441,9261,196,0,0,44.4636,10.0868,34.2525
3,0,38,12,0,0,1,0,23.039,25.1308,25.5647,1444,54872,144,0,0,276.468,530.796,631.555
4,0,48,6,0,0,1,1,24.9944,25.2436,25.5647,2304,110592,36,0,0,149.966,624.718,637.237
5,0,18,8,0,0,1,1,1.6693,10.7276,9.86087,324,5832,64,0,0,13.3544,2.78655,115.082
6,0,22,11,0,0,1,1,16.3658,18.4493,25.5647,484,10648,121,0,0,180.023,267.838,340.376
7,0,48,10,0,0,1,1,16.8046,16.3546,18.0593,2304,110592,100,0,0,168.046,282.396,267.473
8,0,18,11,0,0,0,1,1.14421,3.62003,15.7393,324,5832,121,0,0,12.5863,1.30922,13.1046
9,0,48,9,0,0,1,1,25.8623,25.2436,25.5647,2304,110592,81,0,0,232.761,668.86,637.237


Stadistics

In [86]:
variables_compare = [:age, :educ, :black]

3-element Vector{Symbol}:
 :age
 :educ
 :black

In [87]:
describe(treatment[!,variables_compare])

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Int64,Float64,Int64,Int64,DataType
1,age,25.8162,17,25.0,48,0,Int64
2,educ,10.3459,4,11.0,16,0,Int64
3,black,0.843243,0,1.0,1,0,Int64


In [88]:
describe(control[!,variables_compare])

Row,variable,mean,min,median,max,nmissing,eltype
,Symbol,Float64,Int64,Float64,Int64,Int64,DataType
1,age,33.2252,16,31.0,55,0,Int64
2,educ,12.0275,0,12.0,18,0,Int64
3,black,0.0735368,0,0.0,1,0,Int64


Las personas que han formado parte del programa de capacitación laboral tienen una edad promedio de 25 años (edad objetivo), mientras que el grupo de control tienen una edad aproximada de 33 años. Por otro lado, las personas tratadas tienen 10 años de educación en promedio, mientras que el grupo de control tiene 12 años de educación en promedio. 

3. 

In [89]:
model= lm(@formula(re78 ~ treat), data)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

re78 ~ 1 + treat

Coefficients:
─────────────────────────────────────────────────────────────────────────
                Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────
(Intercept)  14.8467    0.0761429  194.98    <1e-99   14.6974    14.9959
treat        -8.49752   0.712021   -11.93    <1e-31   -9.89316   -7.10188
─────────────────────────────────────────────────────────────────────────

Se puede observar que existe un efecto inverso del tratamiento sobre la población tratada. La aplicación del programa aparantemente ha reducido los ingresos reales de la población de 1978 en un 8.4 por ciento. 

4. 

In [90]:
data

Row,treat,age,educ,black,hisp,marr,nodegree,re74,re75,re78,agesq,agecube,educsq,u74,u75,interaction1,re74sq,re75sq
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,0,45,11,0,0,1,1,21.5167,25.2436,25.5647,2025,91125,121,0,0,236.683,462.967,637.237
2,0,21,14,0,0,0,0,3.17597,5.85256,13.4961,441,9261,196,0,0,44.4636,10.0868,34.2525
3,0,38,12,0,0,1,0,23.039,25.1308,25.5647,1444,54872,144,0,0,276.468,530.796,631.555
4,0,48,6,0,0,1,1,24.9944,25.2436,25.5647,2304,110592,36,0,0,149.966,624.718,637.237
5,0,18,8,0,0,1,1,1.6693,10.7276,9.86087,324,5832,64,0,0,13.3544,2.78655,115.082
6,0,22,11,0,0,1,1,16.3658,18.4493,25.5647,484,10648,121,0,0,180.023,267.838,340.376
7,0,48,10,0,0,1,1,16.8046,16.3546,18.0593,2304,110592,100,0,0,168.046,282.396,267.473
8,0,18,11,0,0,0,1,1.14421,3.62003,15.7393,324,5832,121,0,0,12.5863,1.30922,13.1046
9,0,48,9,0,0,1,1,25.8623,25.2436,25.5647,2304,110592,81,0,0,232.761,668.86,637.237


Features setting 

In [91]:
y= data[!,:re78]

16177-element Vector{Float64}:
 25.564669921875
 13.496080078125
 25.564669921875
 25.564669921875
  9.860869140625
 25.564669921875
 18.05930078125
 15.73926953125
 25.564669921875
  3.92484204101563
 25.564669921875
  6.84778515625
 25.564669921875
  ⋮
  0.0
  0.672877319335938
  0.0
 10.092830078125
  6.28143310546875
 12.5907099609375
  5.11201416015625
 15.952599609375
 36.64694921875
 12.8039697265625
  3.7866279296875
  4.18194189453125

In [92]:
d = data[!,:treat];
x = data[!,2:end];

In [93]:
coerce!(x, Count => Continuous)

Row,age,educ,black,hisp,marr,nodegree,re74,re75,re78,agesq,agecube,educsq,u74,u75,interaction1,re74sq,re75sq
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,45.0,11.0,0.0,0.0,1.0,1.0,21.5167,25.2436,25.5647,2025.0,91125.0,121.0,0.0,0.0,236.683,462.967,637.237
2,21.0,14.0,0.0,0.0,0.0,0.0,3.17597,5.85256,13.4961,441.0,9261.0,196.0,0.0,0.0,44.4636,10.0868,34.2525
3,38.0,12.0,0.0,0.0,1.0,0.0,23.039,25.1308,25.5647,1444.0,54872.0,144.0,0.0,0.0,276.468,530.796,631.555
4,48.0,6.0,0.0,0.0,1.0,1.0,24.9944,25.2436,25.5647,2304.0,110592.0,36.0,0.0,0.0,149.966,624.718,637.237
5,18.0,8.0,0.0,0.0,1.0,1.0,1.6693,10.7276,9.86087,324.0,5832.0,64.0,0.0,0.0,13.3544,2.78655,115.082
6,22.0,11.0,0.0,0.0,1.0,1.0,16.3658,18.4493,25.5647,484.0,10648.0,121.0,0.0,0.0,180.023,267.838,340.376
7,48.0,10.0,0.0,0.0,1.0,1.0,16.8046,16.3546,18.0593,2304.0,110592.0,100.0,0.0,0.0,168.046,282.396,267.473
8,18.0,11.0,0.0,0.0,0.0,1.0,1.14421,3.62003,15.7393,324.0,5832.0,121.0,0.0,0.0,12.5863,1.30922,13.1046
9,48.0,9.0,0.0,0.0,1.0,1.0,25.8623,25.2436,25.5647,2304.0,110592.0,81.0,0.0,0.0,232.761,668.86,637.237


In [94]:
println("\n length of y is \n", size(y,1) )
println("\n num features x is \n", size(x,2 ) )

# Naive OLS
print( "\n Naive OLS that uses all features w/o cross-fitting \n" )
fm = term(:re78) ~ term(:treat) +sum(term.(Symbol.(names(data[:,4:size(data,2)]))));
lres = reg(data, fm);
first(DataFrame(GLM.coeftable(lres)))


 length of y is 
16177

 num features x is 
17

 Naive OLS that uses all features w/o cross-fitting 


Row,Name,Estimate,Std. Error,t-stat,Pr(>|t|),Lower 95%,Upper 95%
,String,Float64,Float64,Float64,Float64,Float64,Float64
1,treat,2.75176e-13,1.83225e-13,1.50184,0.133157,-8.39662e-14,6.34317e-13


In [95]:
function training_sample_append(cv_split, test_sample_index)
    training_indices = []
    for vector in cv_split[Not(test_sample_index)]
            training_indices = [training_indices; vector]
    end
    return training_indices, cv_split[test_sample_index]
end

function dml(x, d, y, dreg, yreg, nfold)
    n = length(y)
    cv = [partition(eachindex(y), fill(1/nfold, nfold-1)..., shuffle = true, rng = 1234)...]
    machine_y = machine(yreg, x, y, scitype_check_level=0)
    machine_d = machine(dreg, x, d, scitype_check_level=0)
    y_hat = zeros(n)
    d_hat = zeros(n)

    for fold in 1:nfold
            training_fold, test_fold = training_sample_append(cv, fold)
            y_hat[test_fold] = MLJ.predict(MLJ.fit!(machine_y, rows = training_fold), x[test_fold, :])
            d_hat[test_fold] = MLJ.predict(MLJ.fit!(machine_d, rows = training_fold), x[test_fold, :])
    end

    resy = y .- y_hat
    resd = reshape(d .- d_hat, (n, 1))
    estimate = lm(resd, resy)
    coef_est = GLM.coef(estimate)[1]
    se = GLM.coeftable(estimate).cols[2][1]
    println(" coef (se) = ", coef_est ,"(",se,")")
    return coef_est, se, resy, resd;
end

function summarize(point, stderr, resy, resd, name)
    return DataFrame(
            model = [name],
            estimate = [point], stderr = [stderr], 
            rmse_y = [sqrt(mean(resy .^ 2))], 
            rmse_d = [sqrt(mean(resd .^ 2))]
    )
end

summarize (generic function with 1 method)

In [96]:
LinearRegressor = @load LinearRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg = Standardizer() |> LinearRegressor()
yreg = Standardizer() |> LinearRegressor()
result_ols = dml(x, d, y, dreg, yreg, 10);
table_ols = summarize(result_ols..., "OLS")

LassoCVRegressor = @load LassoCVRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg = Standardizer() |> LassoCVRegressor(max_iter=200000)
yreg = Standardizer() |> LassoCVRegressor(max_iter=200000)
results_lasso = dml(x, d, y, dreg, yreg, 10);
table_lasso = summarize(results_lasso..., "LassoCV")

RandomForestRegressor = @load RandomForestRegressor pkg=MLJScikitLearnInterface verbosity=0
dreg = RandomForestRegressor()
yreg = RandomForestRegressor()
results_rf = dml(x, d, y, dreg, yreg, 10);
table_rf = summarize(results_rf..., "RF");

dreg = Standardizer() |> LassoCVRegressor(max_iter=200000)
results_mix = dml(x, d, y, dreg , yreg, 10)
table_mix = summarize(results_mix..., "RF/LassoCV");

[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).


 coef (se) = -1.3169513987065712e-15

[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:linear_regressor, …).


(1.0676631473588983e-15)


[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regre

 coef (se) = 2.7127351865314923e-5(0.0007697256409201347)


[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(

 coef (se) = 0.08908705216163758(0.02071794705999632)


[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[], …), …), …).
[ Info: Training machine(:standardizer, …).
[ Info: Training machine(:lasso_cv_regressor, …).
[ Info: Training machine(RandomForestRegressor(n_estimators = 100, …), …).
[ Info: Training machine(DeterministicPipeline(standardizer = Standardizer(features = Symbol[]

 coef (se) = 0.20641800908835187(0.01788454879618417)


In [97]:
pretty_table([table_ols; table_lasso; table_rf; table_mix])

┌────────────┬──────────────┬─────────────┬─────────────┬───────────┐
│      model │     estimate │      stderr │      rmse_y │    rmse_d │
│     String │      Float64 │     Float64 │     Float64 │   Float64 │
├────────────┼──────────────┼─────────────┼─────────────┼───────────┤
│        OLS │ -1.31695e-15 │ 1.06766e-15 │ 1.33962e-14 │ 0.0986487 │
│    LassoCV │   2.71274e-5 │ 0.000769726 │  0.00967164 │ 0.0987935 │
│         RF │    0.0890871 │   0.0207179 │    0.228781 │  0.086774 │
│ RF/LassoCV │     0.206418 │   0.0178845 │    0.225644 │ 0.0987935 │
└────────────┴──────────────┴─────────────┴─────────────┴───────────┘


In [ ]:
Aplicando el método de double debiased machine learning podemos encontrar un efecto positivo robusto (en términos de menor desviaciación estándar) por parte del programa de apoyo laboral. 